In [ ]:
import sys
import os
import random

# Add the absolute path to the src directory 
os.chdir('../src')
src_path = os.path.abspath('../src/')
sys.path.insert(0, src_path)

from setup import *
seed = 123
import random
torch.manual_seed(seed)
random.seed(seed)
np.random.seed(seed)

# GPS - Peptides Func

In [ ]:
mp.set_start_method('spawn', force=True)

# Load entire Peptides-func dataset
train_loader, test_loader, num_classes = load_peptides_func(parallel=True, subset_ratio=1.0)
print('Data done loading')
num_node_features = 9 

# Define model, optimizer, and scheduler
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = GPSLongRange(
    num_node_features=num_node_features,
    hidden_channels=128,
    num_classes=num_classes,
    num_layers=3
).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=5)
print('Starting training')

# Training model, print final results
for epoch in range(50):
    train_loss = gps_longrange_train(model, train_loader, optimizer, device)
    train_acc = gps_longrange_test(model, train_loader, device)
    test_acc = gps_longrange_test(model, test_loader, device)
    if epoch % 10 == 0:
        print(f"EPOCH {epoch} : Train Loss = {train_loss:.4f}, Train Acc = {train_acc:.4f} , Test Acc = {test_acc:.4f}")
        
    scheduler.step(train_loss)
print(f"FINAL: Train Loss = {train_loss:.4f}, Train Acc = {train_acc:.4f} , Test Acc = {test_acc:.4f}")